In [ ]:
import sys
sys.path.append("../src/")
import os
from nnet import CRNN
from pl_trainer import SEDTask4_2021
from utils.encoder import ManyHotEncoder
from utils.utils import batched_decode_preds
import scipy
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import torchaudio
import librosa
import librosa.display
import time
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import h5py
from IPython.display import Audio
from processing.sampler import ConcatDatasetBatchSampler
import torch
import tensorflow as tf
%matplotlib inline

In [ ]:
from processing.datasets import ConcatDatasetUrban, HDF5_dataset

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
import openl3
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'

In [ ]:
from hear21passt.base2levelmel import load_model, get_scene_embeddings, get_timestamp_embeddings

In [ ]:
with open("../config/sed_hear.yaml", "r") as f:
        conf = yaml.safe_load(f)

###  PASST model

In [ ]:
model_passt = load_model()
model_passt = model_passt.cuda()
model_passt.eval()

### torchopenl3 model

In [ ]:
resampler = torchaudio.transforms.Resample(32000, 48000).cuda()
model_openl3 = torchopenl3.models.load_audio_embedding_model(input_repr="mel256", content_type="env",
                                                 embedding_size=6144).cuda()

In [ ]:
with torch.no_grad():
    embed, time_stamps = get_timestamp_embeddings(audio, model_passt)
#embed = get_scene_embeddings(audio, model_passt)
print(embed.shape)
print(time_stamps.shape)

In [ ]:
emb, ts = torchopenl3.get_audio_embedding(audio_res, sr=48000, model=model, hop_size=0.0615)

### Computing embeddings in HDF5

In [ ]:
hf.close()
hf_audio.close()

In [ ]:
type_hf = "train"

hf_audio = h5py.File(conf["data"]["root_path"] + "train.h5", 'r+')
hf = h5py.File(conf["data"]["root_path"] + conf["data"][f"hdf5_{type_hf}"], 'r+')

In [ ]:
type_hf = "train"
dset = "SINGA-PURA"
hf = h5py.File(conf["data"]["root_path"] + conf["data"][f"hdf5_{type_hf}"], 'r')
embeds = hf[dset]["open_l3_512"]

In [ ]:
for i in tqdm(range(embeds.shape[0])):
    print(np.sum(embeds[i]))
    assert np.sum(embeds[i]) > 0

In [ ]:
to_show = "SINGA-PURA"
for d in hf[to_show]:
    print(d)

In [ ]:
dset = "SINGA-PURA"
audio = hf_audio[dset]["audio_32k"] if type_hf == "train" else hf[dset]["audio_32k"]
group = hf[dset]

In [ ]:
embed_size = 512
model_openl3 = openl3.models.load_audio_embedding_model(input_repr="mel256", content_type="env",embedding_size=embed_size, frontend='kapre')

In [ ]:
emb, ts = openl3.get_audio_embedding(audio[0], sr=32000, model=model_openl3, hop_size=0.0615, batch_size=5)

In [ ]:
n_frames = 156
name = "open_l3_" + str(embed_size)
print("embed_size :", embed_size)
print("n_frames :", n_frames)
print("name :", name)
print("group :", group)
sure = input(f"are you sure ? (y/N)")
if sure == "y":
    if name not in group:
        group.create_dataset(name, (audio.shape[0], n_frames, embed_size,))
    embeddings_arr = group[name]

### openl3 computation

In [ ]:
checkpoint = 0

In [ ]:
for i in tqdm(range(checkpoint, audio.shape[0])):
    emb, ts = openl3.get_audio_embedding(audio[i], sr=32000, model=model_openl3, hop_size=0.0615, verbose=False)
    embeddings_arr[i] = emb

### Torchopenl3 computation

In [ ]:
batch_size = 30
assert audio.shape[0] % batch_size
for k in tqdm(range(audio.shape[0]//batch_size+1)):
    audio_batch = torch.from_numpy(audio[k*batch_size:(k+1)*batch_size]).cuda()
    with torch.no_grad():
        audio_res = resampler(audio_batch).cuda()
        emb_openl3, ts_openl3 = torchopenl3.get_audio_embedding(audio_res, sr=48000, model=model_openl3, hop_size=0.0615)
    emb_cpu = emb_openl3.cpu().numpy()
    embeddings_arr[k*batch_size:(k+1)*batch_size] = emb_cpu

In [ ]:
batch_size = 30
assert audio.shape[0] % batch_size
for k in tqdm(range(audio.shape[0]//batch_size+1)):
    audio_batch = torch.from_numpy(audio[k*batch_size:(k+1)*batch_size]).cuda()
    with torch.no_grad():
        emb_openl3, ts_openl3 = torchopenl3.get_audio_embedding(audio_res, sr=32000, model=model_openl3, hop_size=0.0615)
    emb_cpu = emb_openl3.cpu().numpy()
    embeddings_arr[k*batch_size:(k+1)*batch_size] = emb_cpu
group_sgp.close()

In [ ]:
hf.close()

In [ ]:
audio_res = resampler(audio_batch).cuda()

In [ ]:
emb_openl3, ts_openl3 = torchopenl3.get_audio_embedding(audio_res, sr=48000, model=model_openl3, hop_size=0.0615)

In [ ]:
emb_openl3.shape

In [ ]:
encoder.decode_strong(hf["SINGA-PURA"]["groundtruth"][taxo_name][3].T)

In [ ]:
group = hf["SINGA-PURA"]

In [ ]:
ids = SONYC_test_set.ids
print(len(ids))
print(len(group["audio_32k"]))

In [ ]:
if len(ids) == len(group["audio_32k"]):
    print("ouais")
    group.create_dataset("filenames", data=ids)

In [ ]:
hf_test["filenames_sg"][0:10].astype(str)[0]

In [ ]:
for f in hf["SONYC"]["groundtruth"]:
    print(hf["SONYC"]["groundtruth"][f][9])

In [ ]:
taxo_name = "taxonomy_coarse_SONYC"
with open(f"../config/{taxo_name}.yaml", "r") as f:
        taxonomy = yaml.safe_load(f)

In [ ]:
encoder = ManyHotEncoder(
        taxonomy,
        audio_len=conf["data"]["audio_max_len"],
        frame_len=conf["features"]["n_filters"],
        frame_hop=conf["features"]["hop_length"],
        net_pooling=conf["data"]["net_subsample"],
        fs=conf["data"]["fs"],
    )

dataset = HDF5_SONYC_Dataset(
    conf["data"]["root_path"] + conf["data"][f"hdf5_{type_hf}"],
    conf["data"]["root_path"] + conf["data"][f"sonyc_csv_{type_hf}"],
    encoder,
    return_filename=True
)

In [ ]:
print(set(taxonomy["class_labels"]))
print(set(taxonomy["SONYC"].values()))
print(set(taxonomy["SINGA-PURA"].values()))
print(len(taxonomy["class_labels"]))
print(len(set(taxonomy["SONYC"].values())))
print(len(set(taxonomy["SINGA-PURA"].values())))

In [ ]:
labels = np.zeros((len(dataset), len(taxonomy["class_labels"]),))
i = 0
for batch in dataset:
    labels[i] = batch[1]
    i+=1

In [ ]:
print(len(dataset))
print(batch[1])
print(encoder.decode_weak(batch[1]))
print(labels.shape)

In [ ]:
group = hf["SONYC"]
group.create_dataset(f"groundtruth/{taxo_name}", data=labels)

In [ ]:
hf[f"SONYC"]["groundtruth"]

In [ ]:
hf.close()

In [ ]:
SINGAPURA_train_set.ids[0]

In [ ]:
SINGAPURA_train_set.ids[500]

In [ ]:
i=0
for batch in SINGAPURA_train_set:
    assert batch[1].shape == (8, 156)

In [ ]:
test = np.zeros((8,9))
test = test.transpose()
test.shape

In [ ]:
SINGAPURA_val_set = HDF5_SINGAPURA_labelled(
    conf["data"]["root_path"] + conf["data"]["hdf5_val"],
    conf["data"]["root_path"] + conf["data"]["singa-pura_csv_val"],
    encoder,
    return_filename=True,
    taxonomy=taxonomy
)

In [ ]:
SONYC_train_set = HDF5_SONYC_Dataset(
    conf["data"]["root_path"] + conf["data"]["hdf5_train"],
    conf["data"]["root_path"] + conf["data"]["sonyc_csv_train"],
    encoder,
    return_filename=True
)

In [ ]:
SONYC_val_set = HDF5_SONYC_Dataset(
    conf["data"]["root_path"] + conf["data"]["hdf5_val"],
    conf["data"]["root_path"] + conf["data"]["sonyc_csv_val"],
    encoder,
    return_filename=True
)

In [ ]:
SINGAPURA_test_set = HDF5_SINGAPURA_labelled(
    conf["data"]["root_path"] + conf["data"]["hdf5_test"],
    conf["data"]["root_path"] + conf["data"]["singa-pura_csv_test"],
    encoder,
    return_filename=True,
)

SONYC_test_set = HDF5_SONYC_Dataset(
    conf["data"]["root_path"] + conf["data"]["hdf5_test"],
    conf["data"]["root_path"] + conf["data"]["sonyc_csv_test"],
    encoder,
    return_filename=True,
)
test_dataset = torch.utils.data.ConcatDataset([SINGAPURA_test_set, SONYC_test_set])


In [ ]:
test_dataset.datasets[0] == SINGAPURA_test_set

In [ ]:
durations, groundtruths = SINGAPURA_val_set._generate_eval_dfs(taxonomy)

In [ ]:
SINGAPURA_val_set.groundtruths

In [ ]:
df_sonyc_train = pd.read_csv("/tsi/doctorants/fangulo/Detection-Urban/data/metadata/train/SONYC_train.csv")
df_sonyc_val = pd.read_csv("/tsi/doctorants/fangulo/Detection-Urban/data/metadata/val/SONYC_val.csv")

In [ ]:
dset = soundata.initialize("singapura", "/tsi/dcase/SINGA-PURA")
ids = dset.clip_ids  # the list of urbansound8k's clip ids
clips = dset.load_clips()  # Load all clips in the dataset

In [ ]:
clips[SINGAPURA_train_set.ids[1]].events.annotations[0].labels

In [ ]:
SINGAPURA_train_set[1][1]

In [ ]:
idx = np.random.randint(400)
audio, labels, filename = SONYC_train_set[idx]
print(idx)
print(labels.shape)
print(labels[:, 0])
print(np.where(labels))
print(filename)
print(df_sonyc_train[df_sonyc_train["filename"]==filename])

In [ ]:
labels_dataset = np.zeros((300,8))
labels[:,0]
labels_dataset[0]=labels[:,0]

In [ ]:
labels_dataset

In [ ]:
for audio, labels, filename in SONYC_train_set:
    print(labels)

In [ ]:
idx = np.random.randint(400)
audio, labels, filename = SINGAPURA_train_set[idx]
print(idx)
print(labels.shape)
print(filename)
print(clips[filename].events.annotations[0].labels)
print(clips[filename].events.annotations[0].intervals[:,0])
print(clips[filename].events.annotations[0].intervals[:,1])
plt.figure(figsize=(10,5))
plt.imshow(labels)
plt.show()

In [ ]:
model = CRNN.CRNN(**conf['net'])

In [ ]:
model

In [ ]:
input_test = torch.ones((24, 128, 626))
print(model(input_test)[1])

### Computing HDF5 SONYC

In [ ]:
with open("../../data/Urban Sound Exploration/datasets_conf.yaml", "r") as f:
        dconf = yaml.safe_load(f)

In [ ]:
def get_subset_split(annotation_data):
    """
    Get indices for train and validation subsets
    Parameters
    ----------
    annotation_data
    Returns
    -------
    train_idxs
    valid_idxs
    """

    # Get the audio filenames and the splits without duplicates
    data = annotation_data[['split', 'audio_filename', 'annotator_id']]\
                          .groupby(by=['split', 'audio_filename'], as_index=False)\
                          .min()\
                          .sort_values('audio_filename')

    train_idxs = []
    valid_idxs = []
    test_idxs = []

    for idx, (_, row) in enumerate(data.iterrows()):
        if row['split'] == 'train':
            train_idxs.append(idx)
        elif row['split'] == 'validate' and row['annotator_id'] <= 0:
            # For validation examples, only use verified annotations
            valid_idxs.append(idx)
        elif row['split'] == 'test' and row['annotator_id'] <= 0:
            # For validation examples, only use verified annotations
            test_idxs.append(idx)

    return np.array(train_idxs), np.array(valid_idxs), np.array(test_idxs)

In [ ]:
annot_sonyc = pd.read_csv(dconf['sonyc']['annotations']).sort_values('audio_filename')
subset_split = get_subset_split(annot_sonyc)

In [ ]:
dtale.show(annot_sonyc.iloc[subset_split[2]])

In [ ]:
data = annot_sonyc[['split', 'audio_filename', 'annotator_id']]\
                          .groupby(by=['split', 'audio_filename'], as_index=False)\
                          .min()\
                          .sort_values('audio_filename')

In [ ]:
data

In [ ]:
train_idxs = []
valid_idxs = []
test_idxs = []

for idx, (_, row) in enumerate(data.iterrows()):
    print(idx)
    if row['split'] == 'train':
        train_idxs.append(idx)
    elif row['split'] == 'validate' and row['annotator_id'] <= 0:
        # For validation examples, only use verified annotations
        valid_idxs.append(idx)
    elif row['split'] == 'test' and row['annotator_id'] <= 0:
        # For validation examples, only use verified annotations
        test_idxs.append(idx)

In [ ]:
def parse_ground_truth(annotation_path, yaml_path, split="validate", type_annot="verified"):
    """
    Parse ground truth annotations from a CSV file containing both fine-level
    and coarse-level predictions (and possibly additional metadata).
    Returns a Pandas DataFrame in which the column names are coarse
    IDs of the form 1, 2, 3 etc.
    Parameters
    ----------
    annotation_path: string
        Path to the CSV file containing predictions.
    yaml_path: string
        Path to the YAML file containing coarse taxonomy.
    Returns
    -------
    gt_df: DataFrame
        Ground truth.
    """
    # Create dictionary to parse tags
    with open(yaml_path, 'r') as stream:
        yaml_dict = yaml.load(stream, Loader=yaml.Loader)

    # Load CSV file into a Pandas DataFrame.
    ann_df = pd.read_csv(annotation_path)
    
    if type_annot == "verified":
        # Restrict to ground truth ("annotator zero").
        gt_df = ann_df[
            (ann_df["annotator_id"]==0) & (ann_df["split"]==split)]
    elif type_annot == "crowdsourced":
        gt_df = ann_df[
            (ann_df["annotator_id"] != 0) & (ann_df["split"]==split)]
    elif type_annot == "all":
        gt_df = ann_df[
            ann_df["split"]==split]
    
    # Rename coarse columns.
    coarse_dict = yaml_dict["coarse"]
    coarse_renaming = {
        "_".join([str(c), coarse_dict[c], "presence"]): str(c)
        for c in coarse_dict}
    gt_df = gt_df.rename(columns=coarse_renaming)

    # Collect tag names as strings and map them to mixed (coarse-fine) ID pairs.
    # The "mixed key" is a hyphenation of the coarse ID and fine ID.
    fine_dict = {}
    for coarse_id in yaml_dict["fine"]:
        for fine_id in yaml_dict["fine"][coarse_id]:
            mixed_key = "-".join([str(coarse_id), str(fine_id)])
            fine_dict[mixed_key] = yaml_dict["fine"][coarse_id][fine_id]

    # Rename fine columns.
    fine_renaming = {"_".join([k, fine_dict[k], "presence"]): k
        for k in fine_dict}
    gt_df = gt_df.rename(columns=fine_renaming)

    # Loop over coarse tags.
    n_samples = len(gt_df)
    coarse_dict = yaml_dict["coarse"]
    for coarse_id in yaml_dict["coarse"]:
        # Construct incomplete fine tag by appending -X to the coarse tag.
        incomplete_tag = str(coarse_id) + "-X"

        # If the incomplete tag is not in the prediction, append a column of zeros.
        # This is the case e.g. for coarse ID 7 ("dogs") which has a single
        # fine-level tag ("7-1_dog-barking-whining") and thus no incomplete
        # tag 7-X.
        if incomplete_tag not in gt_df.columns:
            gt_df[incomplete_tag] = np.zeros((n_samples,)).astype('int')

    # Return output in DataFrame format.
    return gt_df.sort_values('audio_filename')

In [ ]:
dtale.show(parse_ground_truth(dconf['sonyc']['annotations'], dconf['sonyc']['taxonomy']))

### Transforming dataframes into simpler ones 
Audio tagging : (filename, tags) <br>
SED : filename, (onset, offset, label)

In [ ]:
gt_df = parse_ground_truth(dconf['sonyc']['annotations'], dconf['sonyc']['taxonomy'], split="test", type_annot="verified")
dtale.show(gt_df)
                              

In [ ]:
labels = list(gt_df.columns)
print(labels)
print(labels[12:41] + labels[-10:])

In [ ]:
columns = list(gt_df.columns)
labels = columns[12:41] + columns[-10:]
weak_dict_set = defaultdict(set)

for _, row in gt_df.iterrows():
    for l in labels:
        if row[l]:
            weak_dict_set[row['audio_filename']].add(l)

weak_dict = {k:list(v) for k,v in weak_dict_set.items()}
for fname in gt_df["audio_filename"].unique():
    if fname not in weak_dict.keys():
        weak_dict[fname] = '0'

weak_dict_str = {k: ';'.join(v) for k,v in weak_dict.items()}

In [ ]:
df_weak = pd.DataFrame.from_dict({'filename':weak_dict_str.keys(), 'event_labels':weak_dict_str.values()})



In [ ]:
#df_weak.to_csv('../data/metadata/test/SONYC_test.csv')

In [ ]:
labels = []
for idx in ids:
    for l in clips[idx].events.annotations[0].labels:
        if l not in labels:
            labels.append(l)
labels.sort()
print(labels)
#    if '0-3' in clips[idx].events.annotations[0].labels:
#        raise ValueError(f"Filename {idx} has wrong label")

In [ ]:
clips["[b827ebd63759][2020-10-28T00-30-24Z][manual][---][4edbade2d41d5f80e324ee4f10d401c0]"].events.annotations[0].labels

In [ ]:
example_clip = clips[ids[0]]  # Get the first clip

In [ ]:
ids

In [ ]:
clips[ids[0]].clip_id == ids[0]

### Transforming SINGA-PURA dataframes into train/val/test split

In [ ]:
df_sgp = pd.read_csv(conf['singa-pura']['annotations_recording'])
sensor_ids = ['b827eb0ebf2f', 'b827eb7680c5','b827eb3e52b8']
df_sgp_train = df_sgp[(~df_sgp["sensor_id"].isin(sensor_ids)) & ((df_sgp["month"] != 8) | (df_sgp["date"] > 21))].reset_index()
df_sgp_val = df_sgp[(df_sgp["sensor_id"].isin(sensor_ids)) & ((df_sgp["month"] != 8) | (df_sgp["date"] > 21))].reset_index()
df_sgp_test = df_sgp[(df_sgp["date"] <= 21) & (df_sgp["month"]==8)].reset_index()
print(len(df_sgp_train), len(df_sgp_test), len(df_sgp_val))
len(df_sgp_train)+len(df_sgp_test)+len(df_sgp_val)

df_sgp_train["clip_id"] = df_sgp_train["filename"].apply(lambda x : x.replace(".flac", ""))
df_sgp_val["clip_id"] = df_sgp_val["filename"].apply(lambda x : x.replace(".flac", ""))
df_sgp_test["clip_id"] = df_sgp_test["filename"].apply(lambda x : x.replace(".flac", ""))

df_sgp_train["filename"] = df_sgp_train["foldername"] + "/" + df_sgp_train["filename"]
df_sgp_val["filename"] = df_sgp_val["foldername"] + "/" + df_sgp_val["filename"]
df_sgp_test["filename"] = df_sgp_test["foldername"] + "/" + df_sgp_test["filename"]





dtale.show(df_sgp_train[["filename","clip_id"]])

In [ ]:
df_sgp_train[["filename","clip_id"]].to_csv("../data/metadata/train/SINGA-PURA_train.csv", index=False)
df_sgp_val[["filename","clip_id"]].to_csv("../data/metadata/val/SINGA-PURA_val.csv", index=False)
df_sgp_test[["filename","clip_id"]].to_csv("../data/metadata/test/SINGA-PURA_test.csv", index=False)

In [ ]:
idx = np.random.randint(0,500)
onset = clips[df_sgp_train["clip_id"][idx]].events.annotations[0].intervals[:,0]
offset = clips[df_sgp_train["clip_id"][idx]].events.annotations[0].intervals[:,1]
label = clips[df_sgp_train["clip_id"][idx]].events.annotations[0].labels


In [ ]:
pd.DataFrame({"onset":onset, "offset":offset, "label": label})

In [ ]:
df_sgp_train[["filename","clip_id"]].to_csv("/tsi/doctorants/fangulo/Detection-Urban/data/metadata/train/SINGA-PURA_train.csv", index=False)
df_sgp_val[["filename","clip_id"]].to_csv("/tsi/doctorants/fangulo/Detection-Urban/data/metadata/val/SINGA-PURA_val.csv", index=False)
df_sgp_test[["filename","clip_id"]].to_csv("/tsi/doctorants/fangulo/Detection-Urban/data/metadata/test/SINGA-PURA_test.csv", index=False)

In [ ]:
hf = h5py.File("../data/train.h5", "r")

In [ ]:
list(hf["SONYC"].keys())

In [ ]:
list(hf.keys())

In [ ]:
Audio(hf["SONYC"]["audio_32k"][189], rate=32000)

In [ ]:
df_sn = pd.read_csv("../data/metadata/train/SONYC_train.csv")

In [ ]:
df_sn.iloc[[189]]

In [ ]:
df_sn

In [ ]:
df_unlabelled = pd.read_csv("/tsi/dcase/SINGA-PURA/unlabelled_metadata_public_with_folder.csv")
del df_unlabelled["Unnamed: 0"]
df_unlabelled.to_csv("/tsi/dcase/SINGA-PURA/unlabelled_metadata_public_with_folder.csv", index=False)

In [ ]:
dtale.show(df_unlabelled)

In [ ]:
df_unlabelled["filename"] = df_unlabelled["foldername"]

In [ ]:
df_unlabelled["filename"] = df_unlabelled["foldername"] +  "/" + df_unlabelled["filename"]

In [ ]:
df_unlabelled["filename"].to_csv("../data/metadata/train/SINGA-PURA_unlabelled.csv", index=False)

In [ ]:
test.create_dataset("SONYC/audio", data=np.ones((41,50)))
test.close()

In [ ]:
test = h5py.File("test.h5", "r")
test["SONYC"]["melspec"][10]
